<a href="https://colab.research.google.com/github/NikhithaU/Projects/blob/main/FineTune_GPT2_with_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch
!pip install pdfplumber
!pip install datasets
!pip install pillow==8.3.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Pillow-9.5.0-cp39-cp39-manylinux_2_28_x86_64.whl (3.4 MB)
  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.3.2
    Uninstalling Pillow-8.3.2:
      Successfully uninstalled Pillow-8.3.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Pillow-8.3.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.5.0
    Uninstalling Pillow-9.5.0:
      Successfully uninstalled Pillow-9.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1+cu118 requires pillow!=8.3.*,>=5.3.0, but you have pillow 8.3.2 which is incompatible.
pdfplumber 0.9.0 requires Pillow>=9.1, but you have pillow 8.3.2 which is incompatible.


In [ ]:
import pdfplumber

def pdf_to_text(file_path):
    with pdfplumber.open(file_path) as pdf:
        text = "\n".join([page.extract_text() for page in pdf.pages])
    return text

file_path = "/content/Napoleon_ A Biography.pdf"
pdf_text = pdf_to_text(file_path)

import re

def preprocess_text(text):
    # Add any pre-processing steps you need, such as removing special characters, headers, or footers
    cleaned_text = re.sub(r'[^a-zA-Z0-9.,!? \n]+', '', text)
    return cleaned_text

preprocessed_text = preprocess_text(pdf_text)


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

from datasets import Dataset

def create_dataset(text, tokenizer):
    tokenizer.pad_token = tokenizer.eos_token
    tokenized_text = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    dataset = Dataset.from_dict(tokenized_text)
    return dataset


dataset = create_dataset(preprocessed_text, tokenizer)

training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    save_steps=10_000,
    save_total_limit=2,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

trainer.train()



/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=1, training_loss=6.536710262298584, metrics={'train_runtime': 32.0441, 'train_samples_per_second': 0.031, 'train_steps_per_second': 0.031, 'total_flos': 522584064000.0, 'train_loss': 6.536710262298584, 'epoch': 1.0})

In [ ]:

model.save_pretrained("output")
tokenizer.save_pretrained("output")


('output/tokenizer_config.json',
 'output/special_tokens_map.json',
 'output/vocab.json',
 'output/merges.txt',
 'output/added_tokens.json')

In [ ]:
from transformers import pipeline

def generate_answer(prompt, model, tokenizer):
    max_length = 256
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    generated_text = model.generate(input_ids, max_length=max_length, num_return_sequences=1)
    answer = tokenizer.decode(generated_text[0], skip_special_tokens=True)
    return answer

def ask_question(question, model, tokenizer):
    prompt = f"Question: {question}\nAnswer:"
    answer = generate_answer(prompt, model, tokenizer)
    return answer.strip()

# Load the fine-tuned model and tokenizer
model_path = "output"  # The directory where the fine-tuned model is saved
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

# Ask a question
question = "who is napoleon?"
answer = ask_question(question, model, tokenizer)
print("Answer:", answer)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer: Question: who is napoleon?
Answer:
Napoleon was the first of the French to conquer the United States. He was the first to conquer the United States. He was the first to conquer the United States. He was the first to conquer the United States. He was the first to conquer the United States. He was the first to conquer the United States. He was the first to conquer the United States. He was the first to conquer the United States. He was the first to conquer the United States. He was the first to conquer the United States. He was the first to conquer the United States. He was the first to conquer the United States. He was the first to conquer the United States. He was the first to conquer the United States. He was the first to conquer the United States. He was the first to conquer the United States. He was the first to conquer the United States. He was the first to conquer the United States. He was the first to conquer the United States. He was the first to conquer the United State